In [28]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from tqdm import tqdm
from langchain_openai import AzureOpenAIEmbeddings
import json
import numpy as np
import torch
import time
import random

Let's fetch the dataframes.

In [2]:
from data_extractor import Extractor
extractor_instance = Extractor()
extractor_instance.extract(location="./data/results/df_combined.csv")

df_combined = pd.read_csv("data/results/df_combined.csv")
df_combined.sample(n=3)

Comp domain df length: 1912  - VTT domain df length: 4927
Example of text to compare: FiR 1 - FiR 1 is a Triga-type nuclear research reactor located in Otaniemi, Espoo, Finland, decommissioned by VTT after serving since 1962, used for nuclear research, training, and medical radiation therapy. | Developed by VTT Technical Research Centre of Finland Ltd.


,Unnamed: 0.1,Document number,VAT id,relationship description,source id,source type,source english_id,source description,relationship type,target id,target type,target english_id,target description,Link Source Text,Source Text,text_to_compare,Unnamed: 0,Source Company
3221,3221,VTT822,FI04205273,"Natlink Oy, owning the Tracker brand, develope...",cyber-physical hunting safety solutions,Innovation,cyber-physical hunting safety solutions,Innovative solutions developed by Natlink Oy a...,DEVELOPED_BY,FI04205273,Organization,Natlink Oy,Natlink Oy is an organization owning the Track...,https://www.vttresearch.com/fi/uutiset-ja-tari...,Hyppää pääsisältöön Beyond the obvious ...,cyber-physical hunting safety solutions - Inno...,NaN,NaN
2918,2918,VTT747,FI06259056,The EcoSMR innovation was developed with parti...,EcoSMR,Innovation,EcoSMR,A Finnish ecosystem for Small Modular Reactors...,DEVELOPED_BY,FI01966560,Organization,Teollisuuden Voima Oyj,A Finnish energy company and a business partne...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,EcoSMR - A Finnish ecosystem for Small Modular...,NaN,NaN
5581,5581,COMP441,NaN,The PV-KK-S blast valve is developed and offer...,PV-KK-S blast valve,Innovation,PV-KK-S blast valve,An application of the PV-KK blast valve design...,DEVELOPED_BY,FI22223934,Organization,Halton Marine,A company that offers the PV-KK-S blast valve ...,https://www.halton.com/wp-content/uploads/2020...,79\nBLAST VALVE â¢ PV-KK-SPV-KK-S \nAPPLIC...,PV-KK-S blast valve - An application of the PV...,2493.0,HALTON VENTURES OY


In [3]:
from embedding_generator import EmbeddingGenerator, LocalEmbeddingProvider, OpenAIEmbeddingProvider

# Configuration
METHOD = "openai"  # Supports local and openai
MODEL_NAME = "all-mpnet-base-v2"  # For local embeddings
MODEL_KEY = "gpt-4.1-mini"  # For OpenAI embeddings, I think only this works

# File paths
input_file = "data/results/df_combined.csv"
output_file = "data/results/embeddings.csv"

provider = OpenAIEmbeddingProvider(MODEL_KEY) if METHOD == "openai" else LocalEmbeddingProvider(MODEL_NAME)
generator = EmbeddingGenerator(provider)

result_df = generator.generate_embeddings(input_file, output_file)

2025-05-31 14:27:06,148 - INFO - Initialized OpenAI embedding model with key: gpt-4.1-mini
2025-05-31 14:27:06,148 - INFO - Loading data from data/results/df_combined.csv
2025-05-31 14:27:06,496 - INFO - Loaded 6839 rows from input file
2025-05-31 14:27:06,497 - INFO - Prepared 6839 texts for embedding generation
2025-05-31 14:27:06,497 - INFO - Generating embeddings for 6839 texts using OpenAI API
Processing batches:  72%|███████▏  | 50/69 [00:50<00:17,  1.11it/s]2025-05-31 14:27:57,548 - INFO - HTTP Request: POST https://aaltoaihack25-resource.cognitiveservices.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 429 Too Many Requests"
2025-05-31 14:27:57,550 - INFO - Retrying request to /embeddings in 14.000000 seconds
2025-05-31 14:28:12,109 - INFO - HTTP Request: POST https://aaltoaihack25-resource.cognitiveservices.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
Proc

In [7]:
result_df.head()

,Unnamed: 0.1,Document number,VAT id,relationship description,source id,source type,source english_id,source description,relationship type,target id,...,target english_id,target description,Link Source Text,Source Text,text_to_compare,Unnamed: 0,Source Company,embedding,embedding_provider,embedding_dimension
0,0,VTT0,FI10292588,"FiR 1 nuclear research reactor was developed, ...",FiR 1,Innovation,FiR 1,FiR 1 is a Triga-type nuclear research reactor...,DEVELOPED_BY,FI26473754,...,VTT Technical Research Centre of Finland Ltd.,VTT is a Finnish research and innovation partn...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,FiR 1 - FiR 1 is a Triga-type nuclear research...,NaN,NaN,"[-0.018910497426986694, 0.017533520236611366, ...",openai_gpt-4.1-mini,3072
1,1,VTT0,FI10292588,Centre for Nuclear Safety is being developed a...,Centre for Nuclear Safety,Innovation,Centre for Nuclear Safety,A modern research facility under construction ...,DEVELOPED_BY,FI26473754,...,VTT Technical Research Centre of Finland Ltd.,VTT is a Finnish research and innovation partn...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,Centre for Nuclear Safety - A modern research ...,NaN,NaN,"[-0.009751166217029095, 0.006150579079985619, ...",openai_gpt-4.1-mini,3072
2,2,VTT3,FI08932048,The innovation approach 'Beyond the obvious' i...,Beyond the obvious,Innovation,Beyond the obvious,An innovation approach promising to provide so...,DEVELOPED_BY,FI26473754,...,VTT Technical Research Centre of Finland Ltd,A visionary research and innovation partner fo...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,Beyond the obvious - An innovation approach pr...,NaN,NaN,"[-0.02210899069905281, -0.002120873425155878, ...",openai_gpt-4.1-mini,3072
3,3,VTT4,FI01111693,Data-Driven Bioeconomy project is developed by...,Data-Driven Bioeconomy project,Innovation,Data-Driven Bioeconomy project,An innovation using Big Data for sustainable u...,DEVELOPED_BY,FI26473754,...,VTT Technical Research Centre of Finland,A Finnish research and innovation partner work...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,Data-Driven Bioeconomy project - An innovation...,NaN,NaN,"[-0.015033945441246033, 0.031899139285087585, ...",openai_gpt-4.1-mini,3072
4,4,VTT4,FI01111693,Data-Driven Bioeconomy project's forestry pilo...,Data-Driven Bioeconomy project,Innovation,Data-Driven Bioeconomy project,An innovation using Big Data for sustainable u...,DEVELOPED_BY,temp_1141,...,MHG Systems,An organization leading pilots developing fore...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,Data-Driven Bioeconomy project - An innovation...,NaN,NaN,"[-0.014490003697574139, 0.016991155222058296, ...",openai_gpt-4.1-mini,3072


In [31]:
threshold = 0.80

similar_pairs = []

#result_df = result_df.drop_duplicates(subset=["source description"]).reset_index(drop=True)

combined_embeddings = result_df["embedding"].tolist()
combined_texts = result_df["text_to_compare"].tolist()

start_time = time.time()

# Convert embeddings to a tensor
embeddings_tensor = torch.stack([torch.tensor(e) for e in combined_embeddings])

# Compute cosine similarity matrix
cos_sim_matrix = util.cos_sim(embeddings_tensor, embeddings_tensor)

# Get indices of upper triangle (i < j)
triu_indices = torch.triu_indices(cos_sim_matrix.size(0), cos_sim_matrix.size(1), offset=1)

# Get similarity scores for upper triangle
sim_scores = cos_sim_matrix[triu_indices[0], triu_indices[1]]

# Filter by threshold
mask = sim_scores > threshold
i_indices = triu_indices[0][mask].tolist()
j_indices = triu_indices[1][mask].tolist()
scores = sim_scores[mask].tolist()

# Collect similar pairs
similar_pairs = [(i, j, score) for i, j, score in zip(i_indices, j_indices, scores)]


end = time.time()
print(f"Similarity computation took {end - start_time:.2f} seconds")
num_samples = 5
sampled_pairs = random.sample(similar_pairs, min(num_samples, len(similar_pairs)))

for idx, (i, j, score) in enumerate(sampled_pairs[:5]):  # Print first 5 examples
    print(f"Pair {idx+1}:")
    print(f"  Index {i} → {combined_texts[i]}")
    print(f"  Index {j} → {combined_texts[j]}")
    print(f"  Similarity: {score:.4f}\n")

Similarity computation took 0.84 seconds
Pair 1:
  Index 1928 → kvanttitietokone - A quantum computer developed by VTT and IQM, representing a new type of computer that uses qubits for faster computations on certain problems compared to supercomputers. | Developed by Teknologian tutkimuskeskus VTT
  Index 2316 → kvanttitietokoneiden kehittäminen - Development of quantum computers by IQM as a new technological innovation. | Developed by IQM
  Similarity: 0.8048

Pair 2:
  Index 1978 → FI10410900 - Wood-based renewable diesel fuel developed by UPM, tested on buses in Helsinki metropolitan area since 2016. | Developed by UPM BioVerno
  Index 2126 → FI10410900 - UPM is an energy company involved in the development and testing of wood-based renewable diesel fuel UPM BioVerno for public transport in Helsinki. | Developed by UPM
  Similarity: 0.8780

Pair 3:
  Index 1768 → FI10410900 - Renewable diesel fuel produced by UPM Biofuels that reduces tailpipe emissions compared to fossil diesel. | 